<a href="https://colab.research.google.com/github/eriksonferreira/pattern_recongnition/blob/main/bagging_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Codigo passado


In [19]:
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.model_selection import cross_validate
def maisFrequente(y):
  return Counter(y.flat).most_common(1)[0][0]

class Arvore(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.caracteristica, self.valor = sorteiaCaracteristica(X, y)
    maiores = X[:,self.caracteristica] > self.valor
    if sum(maiores)>0 and sum(~maiores)>0:
      self.maiores = Arvore()
      self.maiores.fit(X[maiores,:], y[maiores])
      self.menores = Arvore()
      self.menores.fit(X[~maiores,:], y[~maiores])
    else:
      self.resposta = maisFrequente( y )
    return self
  def predict(self, X):
    y = np.empty((X.shape[0]))
    if hasattr(self, 'resposta'):
      y[:] = self.resposta
    else:
      maiores = X[:, self.caracteristica] > self.valor
      y[maiores] = self.maiores.predict(X[maiores,:])
      y[~maiores] = self.menores.predict(X[~maiores,:])
    return y
from sklearn.ensemble import BaggingClassifier
modelo = BaggingClassifier(Arvore(),
                           n_estimators=100)
results = cross_validate(modelo, X, y)
results['test_score'], np.mean(results['test_score'])

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 668, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py", line 827, in predict
    predicted_probabilitiy = self.predict_proba(X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py", line 865, in predict_proba
    all_proba = Parallel(
  File "/

(array([nan, nan, nan, nan, nan]), nan)

Corrigindo

In [9]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.model_selection import cross_validate
from sklearn.utils import check_random_state
from sklearn.datasets import load_iris

In [10]:
def maisFrequente(y):
    return Counter(y.flat).most_common(1)[0][0]



definindo função de sorteio de feature

In [11]:
def sorteiaCaracteristica(X, y):
    random_state = check_random_state(None)
    caracteristica = random_state.randint(0, X.shape[1])
    valores_possiveis = np.unique(X[:, caracteristica])
    valor = random_state.choice(valores_possiveis)
    return caracteristica, valor

In [12]:
class Arvore(BaseEstimator, ClassifierMixin):
    def __init__(self, random_state=None):
        self.random_state = random_state

    def fit(self, X, y):
        self.caracteristica, self.valor = sorteiaCaracteristica(X, y)
        maiores = X[:, self.caracteristica] > self.valor
        if sum(maiores) > 0 and sum(~maiores) > 0:
            #precisa passar um random state
            self.maiores = Arvore(self.random_state)
            self.maiores.fit(X[maiores, :], y[maiores])
            self.menores = Arvore(self.random_state)
            self.menores.fit(X[~maiores, :], y[~maiores])
        else:
            self.resposta = maisFrequente(y)
        return self

    def predict(self, X):
        #precisa garantir que é um inteiro
        y = np.empty((X.shape[0]), dtype=np.int64)
        if hasattr(self, 'resposta'):
            y[:] = self.resposta
        else:
            maiores = X[:, self.caracteristica] > self.valor
            y[maiores] = self.maiores.predict(X[maiores, :])
            y[~maiores] = self.menores.predict(X[~maiores, :])
        return y

In [13]:
# Exemplo de uso da Arvore com BaggingClassifier
from sklearn.ensemble import BaggingClassifier

In [14]:
# Dados de exemplo, substituir por seus dados reais
X, y = load_iris(return_X_y=True)

In [15]:
modelo = BaggingClassifier(Arvore(),
                           n_estimators=100,
                           random_state=0)

In [16]:
results = cross_validate(modelo, X, y)

In [17]:
print(results['test_score'], np.mean(results['test_score']))

[0.96666667 0.96666667 0.9        0.96666667 1.        ] 0.9600000000000002
